In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

if not 'spark' in locals():
    spark = SparkSession.builder \
        .master("local[*]") \
        .config("spark.driver.memory","64G") \
        .getOrCreate()

spark

# Get Data from S3

First we load the data source containing raw weather measurements from S3. Since the data doesn't follow any well-known format (like CSV or JSON), we load it as raw text data and extract all required information. 

But first let's load a single year, just to get an impression of the data

In [6]:
storageLocation = "s3://dimajix-training/data/weather"

Read in the year 2003 as `text` using the `spark.read.text` method. The data can be found at `storageLocation + "/2003"` and should be stored in a variable called `raw_weather_2003`. Also using `limit` and `toPandas` retrieve the first 10 rows and display them as a Pandas DataFrame.

In [7]:
raw_weather_2003 = spark.read.text(storageLocation + "/2003")
raw_weather_2003.limit(10).toPandas()

,value
0,0494703160256242003010100003+55200-162717SY-MT...
1,0228703160256242003010100174+55200-162730FM-16...
2,044070316025624200301010053C+55200-162717FM-15...
3,0071703160256242003010101009+55200-162717NSRDB...
4,042770316025624200301010153C+55200-162717FM-15...
5,0071703160256242003010102009+55200-162717NSRDB...
6,046870316025624200301010253C+55200-162717FM-15...
7,0071703160256242003010103009+55200-162717NSRDB...
8,041570316025624200301010353C+55200-162717FM-15...
9,0054703160256242003010104009+55200-162717NSRDB...


## Read in all years

Now we read in all years by creating a union. We also add the year as a logical partition column, this will be used later.

In [8]:
from functools import reduce

# Read in all years, store them in an Python array
raw_weather_per_year = [spark.read.text(storageLocation + "/" + str(i)).withColumn("year", f.lit(i)) for i in range(2003,2020)]

# Union all years together
raw_weather = reduce(lambda l,r: l.union(r), raw_weather_per_year)                        

# Display first 10 records
raw_weather.limit(10).toPandas()

,value,year
0,0494703160256242003010100003+55200-162717SY-MT...,2003
1,0228703160256242003010100174+55200-162730FM-16...,2003
2,044070316025624200301010053C+55200-162717FM-15...,2003
3,0071703160256242003010101009+55200-162717NSRDB...,2003
4,042770316025624200301010153C+55200-162717FM-15...,2003
5,0071703160256242003010102009+55200-162717NSRDB...,2003
6,046870316025624200301010253C+55200-162717FM-15...,2003
7,0071703160256242003010103009+55200-162717NSRDB...,2003
8,041570316025624200301010353C+55200-162717FM-15...,2003
9,0054703160256242003010104009+55200-162717NSRDB...,2003


## Extract Information

The raw data is not exactly nice to work with, so we need to extract the relevant information by using appropriate substr operations.

In [9]:
weather = raw_weather.select(
    f.col("year"),
    f.substring(f.col("value"),5,6).alias("usaf"),
    f.substring(f.col("value"),11,5).alias("wban"),
    f.substring(f.col("value"),16,8).alias("date"),
    f.substring(f.col("value"),24,4).alias("time"),
    f.substring(f.col("value"),42,5).alias("report_type"),
    f.substring(f.col("value"),61,3).alias("wind_direction"),
    f.substring(f.col("value"),64,1).alias("wind_direction_qual"),
    f.substring(f.col("value"),65,1).alias("wind_observation"),
    (f.substring(f.col("value"),66,4).cast("float") / f.lit(10.0)).alias("wind_speed"),
    f.substring(f.col("value"),70,1).alias("wind_speed_qual"),
    (f.substring(f.col("value"),88,5).cast("float") / f.lit(10.0)).alias("air_temperature"),
    f.substring(f.col("value"),93,1).alias("air_temperature_qual")
)
    
weather.limit(10).toPandas()

,year,usaf,wban,date,time,report_type,wind_direction,wind_direction_qual,wind_observation,wind_speed,wind_speed_qual,air_temperature,air_temperature_qual
0,2003,703160,25624,20030101,0000,SY-MT,010,5,N,5.2,5,-0.6,5
1,2003,703160,25624,20030101,0017,FM-16,020,1,N,4.6,1,-2.0,1
2,2003,703160,25624,20030101,0053,FM-15,010,5,N,5.2,5,-2.8,5
3,2003,703160,25624,20030101,0100,NSRDB,999,9,9,999.9,9,999.9,9
4,2003,703160,25624,20030101,0153,FM-15,010,5,N,6.2,5,-2.2,5
5,2003,703160,25624,20030101,0200,NSRDB,999,9,9,999.9,9,999.9,9
6,2003,703160,25624,20030101,0253,FM-15,010,5,N,7.2,5,-3.3,5
7,2003,703160,25624,20030101,0300,NSRDB,999,9,9,999.9,9,999.9,9
8,2003,703160,25624,20030101,0353,FM-15,020,5,N,6.2,5,-1.1,5
9,2003,703160,25624,20030101,0400,NSRDB,999,9,9,999.9,9,999.9,9


## Read in Station Metadata

Fortunately station metadata is stored as CSV, so we can directly read that using Sparks `spark.read.csv` mechanisum. The data can be found at `storageLocation + '/isd-history'`.

You should also specify the `DataFrameReader` option `header` to be `True`, this will use the first line of the CSV for creating column names.

Store the result in a variable called `stations` and again print the first 10 lines using the `toPandas()` method.

Note the countries are NOT ISO codes, they are FIPS-codes: https://en.wikipedia.org/wiki/List_of_FIPS_country_codes

In [10]:
stations = spark.read \
    .option("header", True) \
    .csv(storageLocation + "/isd-history")

# Display first 10 records    
stations.limit(10).toPandas()

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
0,007018,99999,WXPOD 7018,None,None,None,+00.000,+000.000,+7018.0,20110309,20130730
1,007026,99999,WXPOD 7026,AF,None,None,+00.000,+000.000,+7026.0,20120713,20170822
2,007070,99999,WXPOD 7070,AF,None,None,+00.000,+000.000,+7070.0,20140923,20150926
3,008260,99999,WXPOD8270,None,None,None,+00.000,+000.000,+0000.0,20050101,20100920
4,008268,99999,WXPOD8278,AF,None,None,+32.950,+065.567,+1156.7,20100519,20120323
5,008307,99999,WXPOD 8318,AF,None,None,+00.000,+000.000,+8318.0,20100421,20100421
6,008411,99999,XM20,None,None,None,None,None,None,20160217,20160217
7,008414,99999,XM18,None,None,None,None,None,None,20160216,20160217
8,008415,99999,XM21,None,None,None,None,None,None,20160217,20160217
9,008418,99999,XM24,None,None,None,None,None,None,20160217,20160217


# Process Data

Now we want to perform a simple analysis on the data: Calculate minimum and maximum wind speed and air temperature per country and year. This needs to be performed in three steps:

1. Load data as DataFrames (already done)
2. Join both DataFrames `weatherData` and `stationData` on the station code (`wban` and `usaf`)
3. Extract year from date (or use existing column)
4. Rename column `ctry` to `country`
5. Group by country and year
6. Aggregate minimum/maximum values for wind speed and air temperature, pay attention to quality! You have to ignore values for which the quality is not "1"!
7. Print the results. And if you like, make some plots.
 
**Again note** the countries are NOT ISO codes, they are FIPS-codes: [https://en.wikipedia.org/wiki/List_of_FIPS_country_codes](https://en.wikipedia.org/wiki/List_of_FIPS_country_codes), so don't be surprised if apparently the temperature of your country seems to be a little off.

**Since processing the full date range may take a considerable amount of time, you might first want to start with a single year. This can be done by temporarily replacing `raw_weather` with `raw_wather_2003`**

In [12]:
df = weather.join(stations, (weather["usaf"] == stations["USAF"]) & (weather["wban"] == stations["WBAN"]))
result = df.groupBy(df.CTRY, df.year).agg(
        f.min(f.when(df["air_temperature_qual"] == 1, df.air_temperature)).alias('min_temp'),
        f.max(f.when(df["air_temperature_qual"] == 1, df.air_temperature)).alias('max_temp'),
        f.min(f.when(df["wind_speed_qual"] == 1, df.wind_speed)).alias('min_wind'),
        f.max(f.when(df["wind_speed_qual"] == 1, df.wind_speed)).alias('max_wind')
    )

pdf = result.toPandas()    
pdf

,CTRY,year,min_temp,max_temp,min_wind,max_wind
0,NL,2003,-14.3,36.0,0.0,33.4
1,US,2003,-44.0,41.0,0.0,35.0
2,BE,2003,-11.3,36.3,0.0,19.0
3,AU,2003,-17.6,37.4,0.0,22.6
4,EZ,2003,-16.0,37.0,0.0,16.5
...,...,...,...,...,...,...
478,CH,2019,13.0,35.0,0.0,14.0
479,AS,2019,2.7,45.8,0.0,13.4
480,RS,2019,-25.9,29.9,0.0,26.0
481,SF,2019,2.7,42.5,0.0,10.8
